In [2]:
# Cell 1: install required packages
!pip install -q --upgrade pip
!pip install -q pandas numpy scikit-learn lightgbm shap matplotlib seaborn joblib
print("Packages installed")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 24.7 MB/s eta 0:00:00
Packages installed


In [1]:
from google.colab import drive
drive.mount('/content/drive')
# Then put your telco.csv in your Drive and refer path like:
# data_path = "/content/drive/MyDrive/path/to/telco.csv"


Mounted at /content/drive


In [4]:
import pandas as pd

data_path = "/content/drive/MyDrive/datasets/telco.csv"
df = pd.read_csv(data_path)

print("Dataset shape:", df.shape)
df.head()


Dataset shape: (7043, 21)


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [7]:
import os
os.makedirs("outputs", exist_ok=True)

import pandas as pd
import numpy as np
import lightgbm as lgb
import shap
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import roc_auc_score
import joblib

# Load dataset
data_path = "/content/drive/MyDrive/datasets/telco.csv"
df = pd.read_csv(data_path)
df.columns = df.columns.str.strip()

# Fix TotalCharges
df["TotalCharges"] = pd.to_numeric(df["TotalCharges"], errors="coerce")

# Convert Y/N to 1/0
df["Churn"] = df["Churn"].map({"Yes":1, "No":0})

# Drop ID
cid = [c for c in df.columns if c.lower()=="customerid"]
if cid:
    df = df.drop(columns=cid)

y = df["Churn"]
X = df.drop(columns=["Churn"])

# Identify categorical / numeric
cat_cols = X.select_dtypes(include=["object","category"]).columns.tolist()
num_cols = X.select_dtypes(include=[np.number]).columns.tolist()

# Impute numerical
num_imp = SimpleImputer(strategy="median")
X[num_cols] = num_imp.fit_transform(X[num_cols])

# Impute + encode categorical
cat_imp = SimpleImputer(strategy="most_frequent")
X[cat_cols] = cat_imp.fit_transform(X[cat_cols])

enc = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)
X[cat_cols] = enc.fit_transform(X[cat_cols])

print("Prepared data:", X.shape)

# ------- LightGBM CV Training (FIXED EARLY STOPPING) -------
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
oof = np.zeros(len(X))
models = []

for fold, (tr, val) in enumerate(skf.split(X, y), 1):
    Xtr, Xval = X.iloc[tr], X.iloc[val]
    ytr, yval = y.iloc[tr], y.iloc[val]

    model = lgb.LGBMClassifier(
        objective="binary",
        n_estimators=1000,
        learning_rate=0.05,
        num_leaves=31,
        n_jobs=-1
    )

    model.fit(
        Xtr, ytr,
        eval_set=[(Xval, yval)],
        eval_metric="auc",
        callbacks=[lgb.early_stopping(50)]   # <-- FIXED
    )

    preds = model.predict_proba(Xval)[:,1]
    oof[val] = preds
    models.append(model)

    print(f"Fold {fold} completed")

auc = roc_auc_score(y, oof)
print("\n🔵 OOF AUC Score:", auc)

pd.Series({"oof_auc": auc}).to_csv("outputs/oof_metrics.csv")


# ----------------- Gini / Gain Importance ------------------
gain = models[-1].booster_.feature_importance(importance_type="gain")
gain_df = pd.DataFrame({
    "feature": X.columns,
    "gain": gain
}).sort_values("gain", ascending=False)

gain_df.to_csv("outputs/gini_gain_importances.csv", index=False)

print("\n🔷 Top 10 Gain Features:")
display(gain_df.head(10))


# ------------------ SHAP Analysis --------------------------
sample = X.sample(n=min(2000, len(X)), random_state=42)
explainer = shap.TreeExplainer(models[-1].booster_)
sv = explainer.shap_values(sample)

if isinstance(sv, list):
    shap_vals = sv[1]
else:
    shap_vals = sv

shap_imp = np.mean(np.abs(shap_vals), axis=0)

shap_df = pd.DataFrame({
    "feature": sample.columns,
    "mean_abs_shap": shap_imp
}).sort_values("mean_abs_shap", ascending=False)

shap_df.to_csv("outputs/shap_global_importance.csv", index=False)

print("\n🟢 Top 10 SHAP Features:")
display(shap_df.head(10))


# ------------------ SHAP PLOTS -----------------------------
plt.figure(figsize=(8,6))
shap.summary_plot(shap_vals, sample, plot_type="bar", show=False)
plt.savefig("outputs/shap_summary_bar.png", dpi=150, bbox_inches="tight")
plt.clf()

plt.figure(figsize=(10,8))
shap.summary_plot(shap_vals, sample, show=False)
plt.savefig("outputs/shap_summary_dot.png", dpi=150, bbox_inches="tight")
plt.clf()

# Dependence plots for top 3
top3 = shap_df["feature"].head(3).tolist()
for f in top3:
    shap.dependence_plot(f, shap_vals, sample, show=False)
    plt.savefig(f"outputs/shap_dependence_{f}.png", dpi=150, bbox_inches="tight")
    plt.clf()

print("\n🎉 All SHAP plots saved in /content/outputs/")


Prepared data: (7043, 19)
[LightGBM] [Info] Number of positive: 1495, number of negative: 4139
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.022297 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 626
[LightGBM] [Info] Number of data points in the train set: 5634, number of used features: 19
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.265353 -> initscore=-1.018328
[LightGBM] [Info] Start training from score -1.018328
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[32]	valid_0's auc: 0.85876	valid_0's binary_logloss: 0.416816
Fold 1 completed
[LightGBM] [Info] Number of positive: 1495, number of negative: 4139
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000578 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory

,feature,gain
14,Contract,10316.248076
4,tenure,2144.107557
17,MonthlyCharges,2047.413685
8,OnlineSecurity,1441.697906
18,TotalCharges,981.285047
11,TechSupport,769.717090
7,InternetService,702.969224
15,PaperlessBilling,296.192262
16,PaymentMethod,252.351150
13,StreamingMovies,221.837199



🟢 Top 10 SHAP Features:


/usr/local/lib/python3.12/dist-packages/shap/explainers/_tree.py:587: UserWarning: LightGBM binary classifier with TreeExplainer shap values output has changed to a list of ndarray
  warnings.warn(


,feature,mean_abs_shap
14,Contract,0.631422
17,MonthlyCharges,0.208518
4,tenure,0.196934
8,OnlineSecurity,0.135617
11,TechSupport,0.106016
18,TotalCharges,0.058715
7,InternetService,0.056809
13,StreamingMovies,0.045154
15,PaperlessBilling,0.039741
6,MultipleLines,0.024675


...

<Figure size 800x910 with 0 Axes>

<Figure size 800x910 with 0 Axes>

<Figure size 750x500 with 0 Axes>

<Figure size 750x500 with 0 Axes>

<Figure size 750x500 with 0 Axes>

In [8]:
!zip -r outputs.zip outputs
from google.colab import files
files.download("outputs.zip")


  adding: outputs/ (stored 0%)
  adding: outputs/shap_dependence_tenure.png (deflated 3%)
  adding: outputs/shap_summary_bar.png (deflated 22%)
  adding: outputs/shap_dependence_Contract.png (deflated 12%)
  adding: outputs/gini_gain_importances.csv (deflated 33%)
  adding: outputs/shap_dependence_MonthlyCharges.png (deflated 3%)
  adding: outputs/shap_global_importance.csv (deflated 36%)
  adding: outputs/oof_metrics.csv (stored 0%)
  adding: outputs/shap_summary_dot.png (deflated 7%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>